In [1]:
import sys; sys.path.append('/home/clair/wwa'); from wwa import *

# **Data pre-processing**

## **GMST**
Downloaded from [climate explorer](https://climexp.knmi.nl/getindices.cgi?WMO=NASAData/giss_al_gl_a_4yrlo&STATION=smoothed_GMST&TYPE=i&id=62f4b5a82fde776a4c64f0ca33646aa0) on November 27th

In [21]:
# load data
gmst = pd.read_csv("data/gmst.txt", comment = "#", sep = " +", engine = "python", names = ["year", "gmst"], index_col = 0)

# convert to anomaly wrt 2023 & only retain years prior to 2023
gmst = gmst.loc[slice(None, "2023")] - gmst.loc[2023]

In [22]:
# specify filename
csv_fnm = "ts/gmst.dat"
if len(csv_fnm.split("/")[-1]) > 65: print("! Filename too long: ", csv_fnm)

# create extra header lines for upload to Climate Explorer 
str1 = "# contact :: smoothed GMST - Horn of Africa flooding 2023, c.barnes22@imperial.ac.uk"
str2 = "# gmst [degC] anomaly wrt 2023 of 4-year smoothed GMST at "+re.sub(".dat", "", csv_fnm.split("/")[-1])
head = "# year gmst"

# make .dat file
! echo "$str1 " > $csv_fnm
! echo "$str2" >> $csv_fnm
! echo "$head" >> $csv_fnm
gmst.to_csv(csv_fnm, sep = " ", mode = "a", header = False)

## **Detrended Nino3.4**
Nino3.4 detrended using mean of SSTs from -20:20 - no scaling applied per calendar month  
[Nino3.4](https://climexp.knmi.nl/getindices.cgi?WMO=NCDCData/ersst_nino3.4a&STATION=NINO3.4&TYPE=i) and [SSTs](https://climexp.knmi.nl/select.cgi?field=ersstv5) downloaded from climate explorer on November 27th  


In [47]:
# get detrended Nino3.4 index from indices on Climate Explorer
nino = decode_times(xr.open_dataset("data/iersst_nino3.4a.nc", decode_times = False))["Nino3.4"]
sst = decode_times(xr.open_dataset("data/iersstv5_0-360E_-20-20N_n.nc", decode_times = False)).sst
nino_detrended = (nino - (sst - sst.sel(time = slice("1980", "2010")).mean())).rename("nino_dt")

# get 3-month mean of detrended Nino3.4
nino_3m = nino_detrended.rolling(time = 3, center = False, min_periods = 1).mean()

In [50]:
# December data not year available; also not clear which lag is most important, so check several.
nino_ond = nino_3m.groupby("time.month")[12]
nino_son = nino_3m.groupby("time.month")[11]
nino_jasond = nino_detrended.rolling(time = 6, center = False, min_periods = 4).mean().groupby("time.month")[12]

In [72]:
# specify filename
csv_fnm = "ts/nino3.4-detrended_JASOND.dat"
ts = nino_jasond

if len(csv_fnm.split("/")[-1]) > 65: print("! Filename too long: ", csv_fnm)

# create extra header lines for upload to Climate Explorer 
str1 = "# contact :: mean detrended Nino3.4 index for JASOND - Horn of Africa flooding 2023, c.barnes22@imperial.ac.uk"
str2 = "# nino34 [degC] November 2023 snapshot of iersst_nino3.4a detrended with iersstv5_0-360E_-20-20N_n_su from KNMI Climate Explorer at "+re.sub(".dat", "", csv_fnm.split("/")[-1])
head = "# year nino34"

# make .dat file
! echo "$str1 " > $csv_fnm
! echo "$str2" >> $csv_fnm
! echo "$head" >> $csv_fnm

ts = ts.assign_coords(time = ts.time.dt.year).rename(time = "year").dropna("year", "any")

ts.to_dataframe().to_csv(csv_fnm, sep = " ", mode = "a", header = False)

In [90]:
# save 3-month mean time series to check cross-correlations in R
nino_3m.to_dataframe().to_csv("data/nino3.4rel_3month.csv")

## **IOD/DMI**
[IOD](https://climexp.knmi.nl/getindices.cgi?WMO=NCDCData/dmi_ersst&STATION=DMI_ERSST&TYPE=i) downloaded from climate explorer on November 27th  


In [83]:
iod = decode_times(xr.open_dataset("data/idmi_ersst.nc", decode_times = False))["diff"].rename("iod")

iod_3m = iod.rolling(time = 3, center = False, min_periods = 1).mean()

# again, test a few aggregations
iod_ond = iod_3m.groupby("time.month")[12]
iod_son = iod_3m.groupby("time.month")[11]
iod_jasond = iod.rolling(time = 6, center = False, min_periods = 4).mean().groupby("time.month")[12]

In [86]:
# specify filename
csv_fnm = "ts/iod_SON.dat"
ts = iod_son

if len(csv_fnm.split("/")[-1]) > 65: print("! Filename too long: ", csv_fnm)

# create extra header lines for upload to Climate Explorer 
str1 = "# contact :: mean Indian Ocean Dipole index for SON - Horn of Africa flooding 2023, c.barnes22@imperial.ac.uk"
str2 = "# iod [degC] November 2023 snapshot of dmi_ersst from KNMI Climate Explorer at "+re.sub(".dat", "", csv_fnm.split("/")[-1])
head = "# year iod"

# make .dat file
! echo "$str1 " > $csv_fnm
! echo "$str2" >> $csv_fnm
! echo "$head" >> $csv_fnm

ts = ts.assign_coords(time = ts.time.dt.year).rename(time = "year").dropna("year", "any")

ts.to_dataframe().to_csv(csv_fnm, sep = " ", mode = "a", header = False)

In [91]:
# save 3-month mean time series to check cross-correlations in R
iod_3m.to_dataframe().to_csv("data/iod_3month.csv")